In [53]:
from molecule_wrapper import init_molecule, try_update, accept_update, get_current_coords, clear_state

from wiggle.geometry import get_pair_rmsd 

import os
from glob import glob
from tqdm import tqdm

import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, cdist

from typing import List, Tuple
import ast



# # Initialize - use 
coords = '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/SASDCP8/SAS_8_3segs/mol1_sub_0_end_xyz.dat'
fp =  '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/SASDCP8/fingerPrint1.dat'
init_molecule(fp, coords)

# # Try update
# coords = try_update(1)
# if coords is not None:
#     print("Update successful")

number of chains 1


True

In [56]:
try_update(1) - get_current_coords()

array([[  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [ -1.75173069,  -2.86196935,  -4.7769417 ],
       [ -4.31278488,  -3.99993951,  -7.63786203],
       [-10.32569813,  -2.94148997,  -7.70753835],
       [-11.74843428,  -3.92837402, -11.8091137 ],
       [-15.13914255,  -3.11672513, -12.23852357],
       [-16.2764694 ,  -2.72648939, -13.27144462],
       [-15.09735505,  -2.93054228,  -8.74100274],
       [-14.75831311,  -3.11420922, -10.74404463],
       [-13.99561644,  -3.75105142,  -8.31279394],
       [-11.15071738,  -6.5841568 ,  -6.98712515],
       [ -4.95796143,  -8.71452

In [54]:
get_current_coords()

array([[-28.439   , -19.476   ,   7.402   ],
       [-26.527   , -20.488   ,   4.245   ],
       [-22.815   , -19.993   ,   4.96    ],
       [-22.032   , -17.477   ,   2.201   ],
       [-18.904   , -19.078   ,   0.776   ],
       [-16.292   , -16.309   ,   0.827   ],
       [-16.398   , -14.586   ,  -2.597   ],
       [-12.636   , -15.312   ,  -2.806   ],
       [-10.202   , -18.094   ,  -1.765   ],
       [ -7.494   , -15.603   ,  -0.658   ],
       [ -7.206   , -11.955   ,   0.454   ],
       [ -3.761   , -10.422   ,  -0.147   ],
       [ -2.872   ,  -6.941   ,   1.07    ],
       [ -0.304   ,  -5.313   ,  -1.224   ],
       [  1.546   ,  -2.055   ,  -0.572   ],
       [  3.009   ,   0.222   ,  -3.262   ],
       [  5.973   ,   2.269   ,  -1.99    ],
       [  8.63    ,   4.46    ,  -3.641   ],
       [  9.38    ,   8.121   ,  -4.292   ],
       [  6.897   ,  10.803   ,  -5.309   ],
       [  6.567   ,  11.082   ,  -9.123   ],
       [  7.848   ,   7.516   ,  -9.974   ],
       [  

In [ ]:
# RIGHT! Lets take her for a spin

# load the data
df_vary = pd.read_csv('/Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/pdb_diversity_varying_sections.csv')

can_change = df_vary['varying_length'] > 0
df_vary = df_vary[can_change].reset_index(drop=True)

# If you have that string in your dataframe
df_vary['varying_sections'] = df_vary['varying_sections'].apply(ast.literal_eval)

In [ ]:
class WritheCalculator:
    def __init__(self):
        self.PI = np.pi
        
    def dbold(self, points: np.ndarray, size: int, i: int, j: int) -> np.ndarray:
        """Calculate difference vector between points[i+1] and points[j+1]."""
        i = i % (size - 1)
        j = j % (size - 1)
        diff = points[i + 1] - points[j + 1]
        return diff.reshape(3)
    
    def te(self, points: np.ndarray, size: int, i: int) -> np.ndarray:
        """Calculate normalized tangent vector."""
        temp = self.dbold(points, size, i, i-1)
        norm = np.linalg.norm(temp)
        return (temp / norm if norm > 0 else np.zeros(3)).reshape(3)
    
    def acos_clip(self, temp: float) -> float:
        """Calculate arccos with clipping to valid domain."""
        if temp < -1.0:
            return self.PI
        elif temp > 1.0:
            return 0
        else:
            return np.arccos(temp)
            
    def mu(self, points: np.ndarray, size: int, i: int, k: int, m: int, j: int) -> float:
        """Calculate local writhing contribution."""
        ti = self.te(points, size, i)
        tj = self.te(points, size, j)
        dkm = self.dbold(points, size, k, m)
        
        ti = ti.reshape(3)
        tj = tj.reshape(3)
        dkm = dkm.reshape(3)
        
        un = np.cross(ti, dkm)
        deux = np.cross(dkm, tj)
        
        un_norm = np.linalg.norm(un)
        deux_norm = np.linalg.norm(deux)
        
        if un_norm < 1e-10 or deux_norm < 1e-10:
            return 0
        
        un = un / un_norm
        deux = deux / deux_norm
        temp = np.dot(un, deux)
        
        cp = np.cross(ti, tj)
        sign_prod = np.dot(dkm, cp)
        
        if abs(sign_prod) < 1e-9:
            return 0.0
        else:
            if sign_prod >= 0:
                return self.acos_clip(temp)
            else:
                return -self.acos_clip(temp)
    
    def wij(self, points: np.ndarray, size: int, i: int, j: int) -> float:
        """Calculate pairwise writhing contribution."""
        if j > i:
            t1 = self.mu(points, size, i, i-1, j-1, j)
            t2 = self.mu(points, size, i, i, j-1, j)
            t3 = self.mu(points, size, i, i-1, j, j)
            t4 = self.mu(points, size, i, i, j, j)
            return t1 - t2 - t3 + t4
        return 0
    
    def calculate_writhe(self, points: np.ndarray) -> float:
        """Calculate total writhe of a curve."""
        points = np.asarray(points).reshape(-1, 3)
        list_size = len(points)
        sig_sum = 0.0
        
        for i in range(list_size - 1):
            for j in range(i + 1, list_size - 1):
                sig_sum += self.wij(points, list_size, i, j)
        
        return sig_sum / (2 * self.PI)

def downsample_coordinates(coordinates: np.ndarray, section_breaks: List[int]) -> np.ndarray:
    """
    Downsample coordinates based on secondary structure sections.
    Takes first point of each section plus last point of final section.
    
    Args:
        coordinates: (N, 3) array of coordinates
        section_breaks: List of indices where sections end
        
    Returns:
        (M, 3) array of downsampled coordinates where M is number of sections + 1
    """
    # Convert to array and ensure proper shape
    coords = np.asarray(coordinates).reshape(-1, 3)
    
    # Get start indices of each section
    section_starts = [0] + [i+1 for i in section_breaks[:-1]]
    
    # Take first point of each section
    downsampled_points = coords[section_starts]
    
    # Add last point of final section
    last_point = coords[section_breaks[-1]].reshape(1, 3)
    
    return np.vstack([downsampled_points, last_point])

def calculate_writhe_penalty(coordinates: np.ndarray, section_breaks: List[int]) -> float:
    """
    Calculate writhe penalty for a molecule using downsampled coordinates
    
    Args:
        coordinates: (N, 3) array of coordinates
        section_breaks: List of indices where sections end
        
    Returns:
        float: The writhe penalty
    """
    writhe_calculator = WritheCalculator()
    
    # Downsample coordinates
    downsampled_coords = downsample_coordinates(coordinates, section_breaks)
    
    # Calculate writhe
    chain_writhe = abs(writhe_calculator.calculate_writhe(downsampled_coords))
    
    # Calculate length and lower bound based on downsampled points
    sec_len = float(len(downsampled_coords))
    lower_bound = pow((sec_len/7.5), 1.6) - 3.0
    
    # Calculate penalty using sigmoid function
    writhe_penalty = 1.0/(1.0 + np.exp(20.0 * (chain_writhe - lower_bound)))
    
    return writhe_penalty, chain_writhe


def find_section_breaks(ss_string: str) -> list:
    """
    Find indices where secondary structure changes.
    Returns list of indices where each section ends (0-based indexing)
    
    Args:
        ss_string: String of secondary structure assignments ('-', 'H', 'S', etc.)
        
    Returns:
        List of indices where sections end
    """
    breaks = []
    current_char = ss_string[0]
    
    # Find all positions where character changes
    for i in range(len(ss_string)):
        if ss_string[i] != current_char:
            breaks.append(i-1)  # End of previous section
            current_char = ss_string[i]
    
    # Add the last position
    breaks.append(len(ss_string)-1)
    
    return breaks

In [ ]:

def check_adjacent_distances(coords: np.ndarray, min_dist: float = 3.65, max_dist: float = 4) -> Tuple[bool, np.ndarray]:
    """
    Check distances between adjacent C-alpha atoms.
    
    Sequential CA-CA distances in proteins are highly conserved:
    - Mean distance ~3.78Å
    - Typical range 3.7-3.9Å
    - Values outside this range usually indicate backbone geometry issues
    
    Args:
        coords: (N,3) numpy array of C-alpha coordinates
        min_dist: Minimum allowed CA-CA distance (default 3.7 Angstroms)
        max_dist: Maximum allowed CA-CA distance (default 3.9 Angstroms)
    
    Returns:
        Tuple[bool, np.ndarray]: 
            - Boolean indicating if any violations found (True = violations present)
    """
    # Calculate distances between adjacent residues
    diffs = coords[1:] - coords[:-1]  # Get coordinate differences
    distances = np.sqrt(np.sum(diffs * diffs, axis=1))  # Calculate distances
    
    # Check for violations
    violations = (distances < min_dist) | (distances > max_dist)
    has_violations = np.any(violations)
    
    # if has_violations:
    #     violation_indices = np.where(violations)[0]
    #     print("\nViolations found:")
    #     for idx in violation_indices:
    #         print(f"Residues {idx}-{idx+1}: {distances[idx]:.3f}Å")

    return has_violations


def check_overlap(coords: np.ndarray, section_breaks: list[int], min_dist: float = 4.0) -> np.ndarray:
    """
    Check for overlapping sections in protein CA backbone using physical constraints.
    Only checks non-sequential sections since sequential ones are handled by CA distance checks.
    
    Args:
        coords: (N,3) numpy array of coordinates
        section_breaks: List of indices where sections end
        min_dist: Minimum allowed distance between non-sequential CAs (default 4.0 Angstroms)
    
    Returns:
        np.ndarray: Indices of sections that overlap with any non-sequential section
    """
    overlapped_secs = []
    
    # Convert section breaks to start and end indices
    section_starts = [0] + [b + 1 for b in section_breaks[:-1]]
    section_ends = section_breaks
    
    # Only need to check sections that are not sequential
    for i in range(len(section_breaks) - 2):  # -2 because we skip next section
        # Get current section coordinates
        curr_section = coords[section_starts[i]:section_ends[i]+1]
        
        # Check against non-sequential later sections
        for j in range(i + 2, len(section_breaks)):
            # Get coordinates for comparison section
            comp_section = coords[section_starts[j]:section_ends[j]+1]
            
            # Calculate minimum distance between sections
            min_dist_sections = np.min(cdist(curr_section, comp_section))
            
            # If minimum distance is less than threshold, section overlaps
            if min_dist_sections < min_dist:
                overlapped_secs.append(i)
                break
                
    return len(np.array(overlapped_secs)) != 0

def mangle_the_shit_out_of_it(breaks, varying_sections, max_attempts_per_section=100):
    """Try to find valid conformation by modifying varying sections."""
    
    # For each varying section
    for section in varying_sections:
        # Try up to max attempts for this section
        for attempt in range(max_attempts_per_section):
            new_coords = try_update(section)

            if new_coords is None:
                return None  # Skip to next attempt if update failed

            # Check both constraints
            adj_valid = check_adjacent_distances(new_coords)
            overlap_valid = check_overlap(new_coords, breaks)
            
            # If both valid, update current coords and move to next section
            if adj_valid or overlap_valid:
                accept_update()
                break
                
    # Return final coordinates regardless of validity
    return get_current_coords()

In [ ]:
import os

rmsd_lst = []

failed_paths = []

# index 363 fucks the shit out of the kernal - not sure at all what the issue is

for i in tqdm(range(df_vary.shape[0])[364:]):

    path = df_vary['pdb_path'][i]
    varying_sections = df_vary['varying_sections'][i]

    coords_file = path + '/Crystal_structure/coordinates1.dat'
    fp_file = path + '/Crystal_structure/fingerPrint1.dat'

    ss = np.genfromtxt(fp_file, str)[-1]
    breaks = find_section_breaks(ss)

    init_molecule(fp_file, coords_file)
    
    ca_coords = get_current_coords()


    mangled_coords = mangle_the_shit_out_of_it(breaks, varying_sections, max_attempts_per_section=100)

    if mangled_coords is None:
        failed_paths.append(path)

    print('completed: ', path)
    if mangled_coords is not None:
        write_dir = path + '/Mangled'
        os.makedirs(write_dir, exist_ok=True)
        np.savetxt(write_dir+'/mangled_coords1.dat', mangled_coords, fmt='%.4f')
        
        rmsd = get_pair_rmsd(ca_coords, mangled_coords)
        rmsd_lst.append(rmsd)

  0%|          | 0/302 [00:00<?, ?it/s]

number of chains 1


  0%|          | 1/302 [00:00<01:04,  4.70it/s]

completed: Failed to find valid conformation for section 7) after 100 attempts - CA distance check failed
 /Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/PDB_modifications/7UVG
number of chains 1


  1%|          | 3/302 [00:00<01:06,  4.51it/s]

...

completed:  /Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/PDB_modifications/1IUY
number of chains 1
completed:  /Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/PDB_modifications/1WGH
number of chains 1


 99%|█████████▉| 300/302 [01:14<00:00,  5.48it/s]

completed:  /Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/PDB_modifications/2LMK
number of chains 1
completed:  /Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/PDB_modifications/5DI5
number of chains 1


100%|██████████| 302/302 [01:14<00:00,  4.03it/s]

completed: Failed to find valid conformation for section 2) after 100 attempts - CA distance check failed
 /Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/PDB_modifications/4XJV
number of chains 1
completed:  /Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/PDB_modifications/4RJX


In [ ]:
mangled_coord_files = glob('/Users/josh/Documents/PhD/NoLooseEnds_Lab/FoXS-carbonara-comparison/Improving_Carbonara/PDB_modifications/*/Mangled/*')

rmsd_val = []
for f in mangled_coord_files:

    coords_m = np.genfromtxt(f)

    og_coords_file = '/'.join(f.split('/')[:-2]) + '/Crystal_structure/coordinates1.dat'
    coords_og = np.genfromtxt(og_coords_file)

    rmsd_val.append(get_pair_rmsd(coords_og, coords_m))


In [ ]:
diff_cond = np.array(rmsd_val) > 6
high_rsmd_mangled_files = np.array(mangled_coord_files)[diff_cond]

path_ = []
fingerP_ = []
og_coord_ = []

for f in high_rsmd_mangled_files:
    path_.append('/'.join(f.split('/')[:-2]))
    fingerP_.append('/'.join(f.split('/')[:-2]) + '/Crystal_structure/fingerPrint1.dat')
    og_coord_.append('/'.join(f.split('/')[:-2]) + '/Crystal_structure/coordinates1.dat')

df_mang = pd.DataFrame(data={'path': path_, 'fingerP': fingerP_, 'og_coord': og_coord_, 'mangled_coord': high_rsmd_mangled_files})

df_mang.to_csv('high_rmsd_mangled_files.csv', index=False)
